In [62]:
import pandas as pd
import ast
from fastapi import FastAPI
import json
import fastparquet

#rows=[]
#with open('steam_games.json', encoding='utf-8-sig') as f:
 #   for line in f.readlines():
  #    rows.append(ast.literal_eval(line))

#df=pd.DataFrame(rows)

app = FastAPI()

In [63]:
# Lee el archivo Parquet y crea un objeto DataFrame
df = fastparquet.ParquetFile('steam_games.parquet').to_pandas()

# Función para obtener los géneros más vendidos
@app.get("/genero/")
def genero(anio: int):
    df_filtrado = df[df['release_date'] == anio]
    if df_filtrado.empty:
        return {"!": f"No hay datos disponibles para el año {anio}"}
    
    generos_count = {}
    for lista_generos in df_filtrado['genres']:
        for genero_juego in lista_generos:
            generos_count[genero_juego] = generos_count.get(genero_juego, 0) + 1
    
    generos_top = dict(sorted(generos_count.items(), key=lambda item: item[1], reverse=True)[:5])
    return generos_top


In [70]:
# Función para obtener los juegos lanzados en un año
@app.get("/juegos/")
def juegos(anio: int):
   # Filtrar el DataFrame por el año deseado
    df_filtrado = df[df['release_date'] == anio]
    if df_filtrado.empty:
        return {"!": f"No hay datos disponibles para el año {anio}"}
    
    return df_filtrado["title"].unique()

In [71]:
juegos(2019)

array(['Raji: An Ancient Epic',
       'The Legendary Player - Make Your Reputation - OPEN BETA',
       'The End of an Age: Fading Remnants'], dtype=object)

In [73]:
# Función para obtener los specs más repetidos
@app.get("/specs/")
def specs(anio: int):
    df_filtrado = df[df['release_date'] == anio]
    if df_filtrado.empty:
        return {"!": f"No hay datos disponibles para el año {anio}"}
    
    specs_count = {}
    for lista_specs in df_filtrado['specs']:
        if lista_specs is not None:
            for spec in lista_specs:
                specs_count[spec] = specs_count.get(spec, 0) + 1
    
    top_specs = dict(sorted(specs_count.items(), key=lambda item: item[1], reverse=True)[:5])
    return {"top_specs": list(top_specs.keys())}


In [74]:
specs(2018)

{'top_specs': ['Single-player',
  'Steam Achievements',
  'Steam Cloud',
  'Full controller support',
  'Steam Trading Cards']}

In [77]:
# Función para obtener la cantidad de juegos lanzados en un año con early access
@app.get("/earlyacces/")
def earlyacces(anio: int):
    df_filtrado = df[df['release_date'] == anio]
    if df_filtrado.empty:
        return {"!": f"No hay datos disponibles para el año {anio}"}
    frecuencias=df_filtrado["early_access"].sum()
    
    return frecuencias

In [78]:
earlyacces(2015)

224

In [81]:
# Función para obtener el análisis de sentimiento

@app.get("/sentiment/")
def sentiment(anio: int):
    df_filtrado = df[df['release_date'] == anio]
    if df_filtrado.empty:
        return {"!": f"No hay datos disponibles para el año {anio}"}
    frecuencia=df_filtrado["sentiment"].value_counts().to_dict()
    return frecuencia


In [82]:
sentiment(2015)

{'Mixed': 828,
 'sin_calificar': 801,
 'Very Positive': 656,
 'Positive': 534,
 'Mostly Positive': 494,
 '1 user reviews': 332,
 '2 user reviews': 264,
 '3 user reviews': 173,
 'Mostly Negative': 164,
 '4 user reviews': 129,
 '6 user reviews': 120,
 '5 user reviews': 119,
 '8 user reviews': 86,
 '7 user reviews': 81,
 '9 user reviews': 77,
 'Overwhelmingly Positive': 53,
 'Negative': 31,
 'Very Negative': 2,
 'Overwhelmingly Negative': 1}

In [ ]:
# Función para obtener los juegos top 5 con mayor metascore
@app.get("/metascore/")
def metascore(anio: int):
    # Filtrar el DataFrame por el año proporcionado
    df_anio = df[df["release_date"].dt.year == anio]

    # Eliminar duplicados basados en la columna "title"
    juegos_unicos = df_anio.drop_duplicates(subset="title")

    # Ordenar los juegos únicos por Metascore de manera descendente
    juegos_ordenados = juegos_unicos.sort_values(by="metascore", ascending=False)

    # Tomar los 5 juegos top con los Metascores más altos
    top_juegos = juegos_ordenados.head(5)

    # Crear una lista de diccionarios con la información de los juegos top
    juegos_top = top_juegos[["title", "metascore"]].to_dict(orient="records")

    return juegos_top

In [ ]:
metascore(2015)

[{'title': 'Grand Theft Auto V', 'metascore': 96.0},
 {'title': 'Divinity: Original Sin - Enhanced Edition', 'metascore': 94.0},
 {'title': 'Undertale', 'metascore': 92.0},
 {'title': 'METAL GEAR SOLID V: THE PHANTOM PAIN', 'metascore': 91.0},
 {'title': 'Pillars of Eternity', 'metascore': 89.0}]